# Getting started

First, we enable the cluster to scale up. Note that if you run an auto-scaling cluster,
Google will suspend your nodes. Make sure to have the experiment prepared before running the commands.

The following is assumed ready:
* GKE/Kubernetes cluster (see also `terraform/terraform_notebook.ipynb`)
    * 2 nodes pools (default for system & dependencies, experiment pool)
* Docker image (including dataset, to speed-up starting experiments).
    * Within a bash shell
        * Make sure to have the `requirements-cpu.txt` installed (or `requirements-gpu.txt (in a virtual venv/conda environment). You can run `pip3 install -r requirements-cpu.txt`
    * First run the extractor (locally) `python3 -m fltk extractor configs/example_cloud_experiment.json`
        *  This downloads datasets to be included in the docker image.
    * Build the container `DOCKER_BUILDKIT=1 docker build --platform linux/amd64 . --tag gcr.io/$PROJECT_ID/fltk`
    * Push to your gcr.io repository `docker push gcr.io/$PROJECT_ID/fltk`


With that setup, first set some variables used throughout the experiment.


In [1]:
##################
### CHANGE ME! ###
##################
PROJECT_ID="test-bed-fltk-group16-mb"
CLUSTER_NAME="fltk-testbed-cluster"
DEFAULT_POOL="default-node-pool"
EXPERIMENT_POOL="medium-fltk-pool-1"
REGION="us-central1-c"

# In case we do not yet have the credentials/kubeconfig
gcloud container clusters get-credentials $CLUSTER_NAME --region $REGION --project $PROJECT_ID

Fetching cluster endpoint and auth data.
kubeconfig entry generated for fltk-testbed-cluster.


Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update



Scale the default-node-pool up.

In [2]:
# These commands might take a while to complete.
gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
     --num-nodes 1 --region $REGION --quiet

Resizing fltk-testbed-cluster...done.


Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-group16-mb/zones/us-central1-c/clusters/fltk-testbed-cluster].


## Preparation
In case you have already tested something or ran another experiment, we have to remove the deployment of the Orchestrator. This will not delete any experiment data, as this persists on one of the ReadWriteMany PVCs.


Currently, the Orchestrator is deployed using a `Deployment` definition, a future version will replace this with a `Deployment` definition, to make this step unnecessary. For experiments this means the following:

1. A single deployment can exist at a single time in a single namespace. This includes 'completed' experiments.
2. For running batches of experiments, a BatchOrchestrator is provided.


ℹ️ This will not remove any data, but if your orchestrator is still/already running experiments, this will stop the deployment. Running training jobs will not be stopped, for this you can use `kubectl`. ConfigMaps created by the Orchestrator (to provide experiment configurations), will not be removed. See the commented code in the cell below.

In [3]:
# If you want to delete all pytorch trainjobs, uncomment the command below.
# kubectl delete pytorchjobs.kubeflow.org --all --namespace test

# If you want to delete all existing configuration map objects in a namespace, run teh command below
# kubectl delete configmaps --all --namespace test

helm uninstall -n test flearner

W1019 09:29:43.780124   11680 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: flearner: release: not found


: 1

## Install extractor

Deploy the TensorBoard service and persistent volumes, required for deployment of the orchestrator's chart.

In [4]:
helm upgrade --install -n test extractor ../charts/extractor -f ../charts/fltk-values.yaml \
    --set provider.projectName=$PROJECT_ID

W1019 09:30:07.643708   11714 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Release "extractor" does not exist. Installing it now.
NAME: extractor
LAST DEPLOYED: Wed Oct 19 09:30:10 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Get the FLTK extractors Tensorboard URL by running:

export POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")
echo http://localhost:6006/
kubectl -n test port-forward $POD_NAME 6006:6006


## Define experiment configuration files

Deployment of experiments is currently done through a Helm Deployment. A future release (™️) will rework this to a Job definition, as this allows to re-use the template more easily.


> The `EXPERIMENT_FILE` will contain the description of the experiments
> The `CLUSTER_CONFIG` will contain shared configurations for logging, Orchestrator configuration and replication information.

In [70]:
# HOMEWORKS
# EXPERIMENT_FILE="/home/mattiacs/Documents/Developer/Edu/TUDelft/Year_4/Quarter_1/CS4215-QPECS/homeworks_2/ex6_configs/distributed_4_node.json"

# PROJECT
# EXPERIMENT_FILE="../configs/federated_tasks/example_arrival_config.json"
# EXPERIMENT_FILE="../configs/distributed_tasks/example_arrival_config.json"
# CLUSTER_CONFIG="../configs/example_cloud_experiment.json"

EXPERIMENT_FILE="../project/configs/distributed/mnist/epoch_10/experiment_10e-6_10.json"
CLUSTER_CONFIG="../project/configs/project_cloud_experiment.json"

## Setup experiment variables
Next, we will deploy the experiments.


We provide a configuration file, `charts/fltk-values.yaml`, in here change the values under the `provider` block. Change `projectName` to your Google Cloud Project ID.

```yaml
provider:
    domain: gcr.io
    projectName: CHANGE_ME!
    imageName: fltk:latest
```

We use the `--set-file` flag for `helm`, as currently, Helm does not support using files outside of the chart root directory (in this case `charts/orchestrator`). Using `--set-file` we can dynamically provide these files. See also issue [here](https://github.com/helm/helm/issues/3276)


In [71]:
helm uninstall -n test experiment-orchestrator
helm install -n test experiment-orchestrator ../charts/orchestrator -f ../charts/fltk-values.yaml \
    --set-file orchestrator.experiment=$EXPERIMENT_FILE,orchestrator.configuration=$CLUSTER_CONFIG \
    --set provider.projectName=$PROJECT_ID

W1019 15:17:22.791882   12370 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled
W1019 15:17:34.653862   12409 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
NAME: experiment-orchestrator
LAST DEPLOYED: Wed Oct 19 15:17:36 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
You successfully launched an experiment configuration on your cluster in test.

N.B. Make sure to collect all data after completing your experiment!
N.B. Re-installing the orchestrator WILL RESULT IN DELETION OF ALL TRAINJOBS and PODS!


In [72]:
# To get logs from the orchestrator
kubectl logs -n test fl-server

W1019 15:40:18.973491   14184 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-19-2022 13:17:42 root         INFO     Loading file config/configuration.fltk.json
No argument path is provided.
10-19-2022 13:17:42 root         INFO     Starting in cluster mode.
10-19-2022 13:17:42 root         INFO     Starting with experiment replication: 0 with seed: 42
10-19-2022 13:17:42 root         INFO     Starting as Orchestrator
10-19-2022 13:17:42 root         INFO     Starting Orchestrator, initializing resources....
10-19-2022 13:17:42 root         INFO     Loading in cluster configuration file
10-19-2022 13:17:42 root         INFO     Pointing configuration to in cluster configuration.
10-19-2022 13:17:42 root         INFO     Starting cluster manager
10-19-2022 13:17:42 ClusterManager INFO     Spinning up cluster manag

10-19-2022 13:20:48 root         INFO     Waiting for 6cc6085b-4f8d-414f-b430-1e504f8a9b6f to complete, 0 pending, 0 arrivals
10-19-2022 13:20:48 ResourceWatchDog INFO     Fetching pod information of cluster...
10-19-2022 13:20:49 ResourceWatchDog INFO     Fetching pod information of cluster...
10-19-2022 13:20:53 root         INFO     Waiting for 6cc6085b-4f8d-414f-b430-1e504f8a9b6f to complete, 0 pending, 0 arrivals
10-19-2022 13:20:58 root         INFO     Waiting for 6cc6085b-4f8d-414f-b430-1e504f8a9b6f to complete, 0 pending, 0 arrivals
10-19-2022 13:20:59 ResourceWatchDog INFO     Fetching pod information of cluster...
10-19-2022 13:21:03 root         INFO     Waiting for 6cc6085b-4f8d-414f-b430-1e504f8a9b6f to complete, 0 pending, 0 arrivals
10-19-2022 13:21:08 root         INFO     Waiting for 6cc6085b-4f8d-414f-b430-1e504f8a9b6f to complete, 0 pending, 0 arrivals
10-19-2022 13:21:09 ResourceWatchDog INFO     Fetching pod information of cluster...
10-19-2022 13:21:13 root      

10-19-2022 13:24:45 root         INFO     Waiting for 6cc6085b-4f8d-414f-b430-1e504f8a9b6f to complete, 0 pending, 0 arrivals
10-19-2022 13:24:47 ResourceWatchDog INFO     Fetching pod information of cluster...
10-19-2022 13:24:50 root         INFO     Waiting for 6cc6085b-4f8d-414f-b430-1e504f8a9b6f to complete, 0 pending, 0 arrivals
10-19-2022 13:24:55 root         INFO     Waiting for 6cc6085b-4f8d-414f-b430-1e504f8a9b6f to complete, 0 pending, 0 arrivals
10-19-2022 13:24:57 ResourceWatchDog INFO     Fetching pod information of cluster...
10-19-2022 13:24:57 ResourceWatchDog INFO     Fetching pod information of cluster...
10-19-2022 13:25:00 root         INFO     Waiting for 6cc6085b-4f8d-414f-b430-1e504f8a9b6f to complete, 0 pending, 0 arrivals
10-19-2022 13:25:05 root         INFO     Waiting for 6cc6085b-4f8d-414f-b430-1e504f8a9b6f to complete, 0 pending, 0 arrivals
10-19-2022 13:25:08 ResourceWatchDog INFO     Fetching pod information of cluster...
10-19-2022 13:25:10 root      

10-19-2022 13:28:41 root         INFO     Waiting for 6cc6085b-4f8d-414f-b430-1e504f8a9b6f to complete, 0 pending, 0 arrivals
10-19-2022 13:28:45 ResourceWatchDog INFO     Fetching pod information of cluster...
10-19-2022 13:28:46 root         INFO     Waiting for 6cc6085b-4f8d-414f-b430-1e504f8a9b6f to complete, 0 pending, 0 arrivals
10-19-2022 13:28:51 root         INFO     Waiting for 6cc6085b-4f8d-414f-b430-1e504f8a9b6f to complete, 0 pending, 0 arrivals
10-19-2022 13:28:56 ResourceWatchDog INFO     Fetching pod information of cluster...
10-19-2022 13:28:56 root         INFO     Waiting for 6cc6085b-4f8d-414f-b430-1e504f8a9b6f to complete, 0 pending, 0 arrivals
10-19-2022 13:29:01 root         INFO     Waiting for 6cc6085b-4f8d-414f-b430-1e504f8a9b6f to complete, 0 pending, 0 arrivals
10-19-2022 13:29:06 root         INFO     Waiting for 6cc6085b-4f8d-414f-b430-1e504f8a9b6f to complete, 0 pending, 0 arrivals
10-19-2022 13:29:06 ResourceWatchDog INFO     Fetching pod information of 

10-19-2022 13:32:37 root         INFO     Waiting for 6cc6085b-4f8d-414f-b430-1e504f8a9b6f to complete, 0 pending, 0 arrivals
10-19-2022 13:32:42 root         INFO     Waiting for 6cc6085b-4f8d-414f-b430-1e504f8a9b6f to complete, 0 pending, 0 arrivals
10-19-2022 13:32:44 ResourceWatchDog INFO     Fetching pod information of cluster...
10-19-2022 13:32:47 root         INFO     Waiting for 6cc6085b-4f8d-414f-b430-1e504f8a9b6f to complete, 0 pending, 0 arrivals
10-19-2022 13:32:52 root         INFO     Waiting for 6cc6085b-4f8d-414f-b430-1e504f8a9b6f to complete, 0 pending, 0 arrivals
10-19-2022 13:32:55 ResourceWatchDog INFO     Fetching pod information of cluster...
10-19-2022 13:32:57 root         INFO     Waiting for 6cc6085b-4f8d-414f-b430-1e504f8a9b6f to complete, 0 pending, 0 arrivals
10-19-2022 13:33:02 root         INFO     Waiting for 6cc6085b-4f8d-414f-b430-1e504f8a9b6f to complete, 0 pending, 0 arrivals
10-19-2022 13:33:05 ResourceWatchDog INFO     Fetching pod information of 

10-19-2022 13:36:33 root         INFO     6cc6085b-4f8d-414f-b430-1e504f8a9b6f was completed with status: Succeeded, moving to completed
10-19-2022 13:36:38 root         INFO     Experiment completed.
10-19-2022 13:36:38 Orchestrator INFO     Received stop signal for the Orchestrator.
10-19-2022 13:36:38 ClusterManager INFO     Stopping execution of ClusterManager, halting components...
10-19-2022 13:36:38 ResourceWatchDog INFO     [WatchDog] Received request to stop execution
10-19-2022 13:36:38 ClusterManager INFO     Successfully stopped execution of ClusterManager
10-19-2022 13:36:38 root         INFO     Stopped execution of Orchestrator replication: 0.


In [73]:
CURRENT_POD_ID="6cc6085b-4f8d-414f-b430-1e504f8a9b6f"
# To get logs from learners (example)
kubectl logs -n test trainjob-$CURRENT_POD_ID-master-0

W1019 15:40:41.822590   14231 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-19-2022 13:17:50 root         INFO     Loading file config/configuration.fltk.json
10-19-2022 13:17:50 root         INFO     Starting in client mode
10-19-2022 13:17:50 root         INFO     Starting with host=trainjob-6cc6085b-4f8d-414f-b430-1e504f8a9b6f-master-0 and port=23456
10-19-2022 13:17:50 root         INFO     Initializing backend for training process: gloo
10-19-2022 13:17:52 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 0
10-19-2022 13:17:52 torch.distributed.distributed_c10d INFO     Rank 0: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
10-19-2022 13:17:52 root         INFO     Starting Creating client with 0
10-19-2022 13:17:52 Client-0-6

       0.1048951 ,        nan, 0.11279826, 0.        ,        nan]), class_recall=array([0.0037037 , 0.        , 0.        , 0.69259259, 0.        ,
       0.13043478, 0.        , 0.21311475, 0.        , 0.        ]), confusion_mat=array([[  1, 159,   0,  50,   2,  48,   0,  10,   0,   0],
       [  0,   0,   0, 233,   0,  32,   0,   3,   0,   0],
       [  0,  90,   0,  72,   0,  15,   0,  92,   0,   0],
       [  0,  33,   0, 187,   0,  34,   0,  16,   0,   0],
       [  0,  87,   0,  60,   0,  33,   0,  73,   2,   0],
       [  0,   4,   0, 169,   1,  30,   0,  25,   1,   0],
       [  0,  98,   0,  41,   0,  36,   0,  45,   1,   0],
       [  0,   0,   0, 185,   3,   4,   0,  52,   0,   0],
       [  0,  73,   0,  44,   4,  51,   0,  64,   0,   0],
       [  0,  38,   0, 115,   0,   3,   0,  81,   0,   0]])), EpochData(epoch_id=4, num_epochs=11, duration_train=436652, duration_test=6793, loss_train=2.5179708051681517, accuracy=10.88, loss=50.093242168426514, class_precision=array([

  return np.diagonal(conf_mat) / np.sum(conf_mat, axis=0)
10-19-2022 13:36:31 root         INFO     Stopping client...
       [  0,  40,   0, 104,   0,   3,   0,  90,   0,   0]]))]


In [74]:
# To get logs from learners (federated learning)
kubectl logs -n test trainjob-$CURRENT_POD_ID-worker-0

W1019 15:41:32.831447   14337 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Defaulted container "pytorch" out of: pytorch, init-pytorch (init)
10-19-2022 13:17:52 root         INFO     Loading file config/configuration.fltk.json
10-19-2022 13:17:52 root         INFO     Starting in client mode
10-19-2022 13:17:52 root         INFO     Starting with host=trainjob-6cc6085b-4f8d-414f-b430-1e504f8a9b6f-master-0 and port=23456
10-19-2022 13:17:52 root         INFO     Initializing backend for training process: gloo
10-19-2022 13:17:52 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 1
10-19-2022 13:17:52 torch.distributed.distributed_c10d INFO     Rank 1: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
10-19-2022 13:17:52 root         INFO

       0.14728682,        nan, 0.09153318, 0.        ,        nan]), class_recall=array([0.        , 0.00743494, 0.        , 0.72373541, 0.        ,
       0.14179104, 0.        , 0.15686275, 0.        , 0.        ]), confusion_mat=array([[  0, 155,   0,  37,   1,  34,   0,  11,   0,   0],
       [  0,   2,   0, 236,   0,  28,   0,   2,   1,   0],
       [  0,  81,   0,  56,   1,  13,   0,  88,   0,   0],
       [  0,  34,   0, 186,   1,  27,   0,   9,   0,   0],
       [  1,  87,   0,  58,   0,  30,   0,  52,   2,   0],
       [  0,   4,   0, 186,   1,  38,   0,  35,   4,   0],
       [  1, 121,   1,  50,   0,  43,   0,  38,   0,   0],
       [  0,   0,   0, 203,   2,  10,   0,  40,   0,   0],
       [  0,  78,   0,  55,   3,  31,   0,  69,   0,   0],
       [  0,  48,   0, 108,   1,   4,   0,  93,   0,   0]])), EpochData(epoch_id=4, num_epochs=11, duration_train=436709, duration_test=8289, loss_train=2.5179708051681517, accuracy=10.72, loss=49.87600541114807, class_precision=array([0

  return np.diagonal(conf_mat) / np.sum(conf_mat, axis=0)
10-19-2022 13:36:30 root         INFO     Stopping client...
       [  0,  52,   0,  91,   1,   6,   0, 104,   0,   0]]))]


In [75]:
kubectl logs -n test trainjob-$CURRENT_POD_ID-worker-1

W1019 15:41:40.810827   14371 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Defaulted container "pytorch" out of: pytorch, init-pytorch (init)
10-19-2022 13:17:52 root         INFO     Loading file config/configuration.fltk.json
10-19-2022 13:17:52 root         INFO     Starting in client mode
10-19-2022 13:17:52 root         INFO     Starting with host=trainjob-6cc6085b-4f8d-414f-b430-1e504f8a9b6f-master-0 and port=23456
10-19-2022 13:17:52 root         INFO     Initializing backend for training process: gloo
10-19-2022 13:17:52 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 2
10-19-2022 13:17:52 torch.distributed.distributed_c10d INFO     Rank 2: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
10-19-2022 13:17:52 root         INFO

       0.11678832,        nan, 0.10752688, 0.        ,        nan]), class_recall=array([0.        , 0.00434783, 0.        , 0.67857143, 0.        ,
       0.13559322, 0.        , 0.19685039, 0.        , 0.        ]), confusion_mat=array([[  0, 143,   0,  47,   1,  38,   0,  14,   1,   0],
       [  0,   1,   0, 186,   0,  39,   0,   3,   1,   0],
       [  0,  77,   0,  54,   2,  12,   0,  89,   0,   0],
       [  0,  26,   0, 171,   2,  37,   0,  16,   0,   0],
       [  0, 100,   0,  66,   0,  31,   0,  63,   1,   0],
       [  0,  12,   0, 164,   0,  32,   0,  28,   0,   0],
       [  0, 110,   0,  66,   1,  35,   0,  57,   0,   0],
       [  0,   0,   0, 197,   3,   4,   0,  50,   0,   0],
       [  0,  98,   0,  56,   8,  43,   0,  66,   0,   0],
       [  0,  37,   0, 130,   0,   3,   0,  79,   0,   0]])), EpochData(epoch_id=4, num_epochs=11, duration_train=436782, duration_test=7500, loss_train=2.5179708051681517, accuracy=10.28, loss=50.59002494812012, class_precision=array([ 

  return np.diagonal(conf_mat) / np.sum(conf_mat, axis=0)
10-19-2022 13:36:31 root         INFO     Stopping client...
       [  0,  41,   1, 111,   0,   3,   0,  93,   0,   0]]))]


## Copy experiment results from the extractor

Extractor holds the experiment results in the format that can be processedby TensorBoard.
In order to download it to the local machine, execute:

In [76]:
EXTRACTOR_POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")

kubectl cp -n test $EXTRACTOR_POD_NAME:/opt/federation-lab/logging ../logging

W1019 15:42:31.888203   14421 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
W1019 15:42:32.776003   14448 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
tar: Removing leading `/' from member names


# Cleanup

## Removing orchestrator

In [77]:
helm uninstall -n test experiment-orchestrator

W1019 15:42:53.020634   14486 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled


## Removing extractor

IMPORTANT: Removing extractor chart will result in deleting the already collected experiment results, stored in the NFS!

In [78]:
helm uninstall extractor -n test

W1019 15:42:58.179859   14502 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "extractor" uninstalled


## Wrapping up

To scale down the cluster nodepools, run the cell below. This will scale the node pools down and remove all the experiments deployed (on the cluster).

1. Experiments cannot be restarted.
2. Experiment logs will not persist deletion.


In [79]:
# This will remove all information and logs as well.
kubectl delete pytorchjobs.kubeflow.org --all-namespaces --all

gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
    --num-nodes 0 --region $REGION --quiet

gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
    --num-nodes 0 --region $REGION --quiet

W1019 15:43:21.797054   14527 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
pytorchjob.kubeflow.org "trainjob-0203a077-858f-4427-ae7f-9c7ad5b92c71" deleted
pytorchjob.kubeflow.org "trainjob-041637b0-9be2-4a36-bc1f-0b8488d1fdd7" deleted
pytorchjob.kubeflow.org "trainjob-0a65e275-2e1f-421a-9f58-b6a809f81e41" deleted
pytorchjob.kubeflow.org "trainjob-1be7dd3d-d117-48aa-a5a2-190743e8d845" deleted
pytorchjob.kubeflow.org "trainjob-20908ee6-abae-4b99-b508-3f36dd9161ee" deleted
pytorchjob.kubeflow.org "trainjob-490cdeec-195f-45b0-94bf-fac30ce4e481" deleted
pytorchjob.kubeflow.org "trainjob-62e17e42-a74b-40b6-932d-568c69ea8c84" deleted
pytorchjob.kubeflow.org "trainjob-62ef1b19-cbb7-49d4-ae48-b74a1f16afa0" deleted
pytorchjob.kubeflow.org "trainjob-6cc6085b-4f8d-414f-b430-1e504f8a9b6f" deleted
pytorchjob.kubeflow.org "trai